In [9]:
# Attempt 1

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt

# Step 1: Load the CSV files
csv12 = '/Users/schoudhry/Desktop/IIT/Research/researchData/StableS12.csv'
csv13 = '/Users/schoudhry/Desktop/IIT/Research/researchData/Stable13.csv'
csv1 = '/Users/schoudhry/Desktop/IIT/Research/researchData/Stable1.csv'

# Load gene expression (S12), bacterial abundance (S13), and tumor/no tumor labels (S1)
data_s12 = pd.read_csv(csv12)
data_s13 = pd.read_csv(csv13)
data_s1 = pd.read_csv(csv1)

# Step 2: Extract relevant columns for B01 to B26
patients = [f'B{str(i).zfill(2)}' for i in range(1, 27)]  # B01 to B26

genes_data = data_s12[['Unnamed: 0'] + patients].rename(columns={'Unnamed: 0': 'Gene'})  # Gene expression
bacteria_data = data_s13[['Unnamed: 0'] + patients].rename(columns={'Unnamed: 0': 'Bacteria'})  # Bacterial abundance

# Step 3: Load the CRC status from csv1 (using 'Description' to label tumor=1 and normal=0)
crc_status = data_s1[['Patient_Blind_ID', 'Description']].query("Patient_Blind_ID in @patients")
crc_status['CRC_Status'] = crc_status['Description'].apply(lambda x: 1 if x == 'tumor' else 0)

# Step 4: Merge gene, bacteria, and CRC status data for all patients (B01 to B26)
merged_data = pd.concat([genes_data.set_index('Gene').T, bacteria_data.set_index('Bacteria').T], axis=1)

# Step 5: Convert all columns to numeric, force errors to NaN
merged_data = merged_data.apply(pd.to_numeric, errors='coerce')

# Step 6: Check for any remaining non-numeric or NaN values
print("Checking for NaN values in the dataset...")
print(merged_data.isnull().sum())  # Check for NaN values

# Handle missing values (fill NaNs with 0 or use a different strategy)
merged_data.fillna(0, inplace=True)

# Merge with CRC status
merged_data['CRC_Status'] = crc_status.set_index('Patient_Blind_ID')['CRC_Status']

# Step 7: Prepare the input features (X) and target (y)
X = merged_data.drop(columns=['CRC_Status']).values  # Features: gene and bacterial data
y = merged_data['CRC_Status'].values  # Target: CRC status (tumor or no tumor)

# Step 8: Normalize the data (optional but recommended)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 9: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Step 10: Define the neural network model
model = tf.keras.Sequential([
    layers.InputLayer(input_shape=(X_train.shape[1],)),  # Input layer

    # First Dense Layer
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),  # Batch normalization for stable learning
    layers.Dropout(0.3),  # Dropout to prevent overfitting

    # Second Dense Layer
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    # Third Dense Layer
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),

    # Fourth Dense Layer
    layers.Dense(32, activation='relu'),

    # Output Layer (sigmoid for binary classification)
    layers.Dense(1, activation='sigmoid')
])

# Step 11: Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 12: Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Step 13: Visualize the model architecture using plot_model
plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)

# Step 14: Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Step 15: Plot training history (accuracy and loss)
plt.figure(figsize=(12, 5))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

plt.show()

# Step 16: Display the saved model plot
from IPython.display import Image
Image('model_architecture.png')  # Show the saved model plot


Checking for NaN values in the dataset...
TSPAN6                                                                                                 0
TNMD                                                                                                   0
DPM1                                                                                                   0
SCYL3                                                                                                  0
C1orf112                                                                                               0
                                                                                                      ..
Bacteria;Actinobacteria;Actinobacteria;Actinomycetales;Mycobacteriaceae;Mycobacterium;arupense         0
Bacteria;Proteobacteria;Gammaproteobacteria;Pasteurellales;Pasteurellaceae;Gallibacterium;genomosp.    0
Bacteria;Actinobacteria;Actinobacteria;Actinomycetales;Mycobacteriaceae;Mycobacterium;celatum          0
Bacteria;Spir

/Users/schoudhry/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [8]:
print(data_s1['Patient_Blind_ID'].unique())


[12337 12423  3600  3792  5691  3566  3874  4771 11808 12379 12372 13796
 14211  1762  1183  1343  1391  1457  1178  2093  1157  2186  2022  1944
  2567  2635  7647  2681  1200  1926  2216  1340  2343  1277  2840  1619
  4571  5741  4044  6470  6645  1552  8567  2853]
